In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import sklearn.metrics as sm
from sklearn.model_selection import cross_validate
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.ensemble import GradientBoostingClassifier as grad
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
import scipy.stats as stats
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.insert(0, "../src/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Grab datingFull.csv

In [2]:
with open('../data/processedData/columnDataDictionary.json') as d:
    columnDataDictionary = json.load(d)
if exists('../data/plotlyDashData/datingFull.csv'):
    datingFull = pd.read_csv('../data/plotlyDashData/datingFull.csv')
else:
    datingFull = pd.read_csv('../data/processedData/datingFull.csv')

# Stringify specified columns

In [3]:
datingFull['zipcode'] = datingFull['zipcode'].apply(str)
for col in list(datingFull.columns):
    if col in columnDataDictionary["nonBinaryCategoricalList"]:
        datingFull[col] = datingFull[col].apply(str)

# Ensure datingFull has locations

In [4]:
if("lats" not in datingFull.columns or "lons" not in datingFull.columns):
    datingFull = util.getLocations(datingFull,True)

# Fix Ambiguous Scores

In [5]:
halfwayChangeColumns = [str(col) for col in datingFull.columns if (("1_s" in str(col)) | ("3_s" in str(col)))]

if(len(halfwayChangeColumns) > 0):
    datingFull = util.fixAmbiguousScores(datingFull)

# Replace Nans and save

In [6]:
datingFullNumerical = datingFull.select_dtypes(include=['uint8','int64','float64']).drop("match",axis=1)
datingFullNumerical = util.replaceNansWithTrainingDataValues(datingFullNumerical)
for col in datingFullNumerical.columns:
    datingFull[col] = datingFullNumerical[col]
datingFull.to_csv('../data/plotlyDashData/datingFull.csv',index=False)